# Reading the Data

In [9]:
import gzip
from collections import defaultdict
import random
import numpy
import scipy.optimize

In [10]:
path = "amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"

In [11]:
f = gzip.open(path, 'rt', encoding = "utf8")

In [12]:
header = f.readline()
header = header.strip().split('\t')
print(header)

['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date']


## Our goal is to make recommendations of products based on users' purchase histories. 

## The only information needed to do so is user and item IDs.

In [13]:
dataset = []

In [14]:
for line in f:
    fields = line.strip().split('\t')
    d = dict(zip(header, fields))
    d['star_rating'] = int(d['star_rating'])
    d['helpful_votes'] = int(d['helpful_votes'])
    d['total_votes'] = int(d['total_votes'])
    dataset.append(d)

In [15]:
dataset[0]

{'marketplace': 'US',
 'customer_id': '45610553',
 'review_id': 'RMDCHWD0Y5OZ9',
 'product_id': 'B00HH62VB6',
 'product_parent': '618218723',
 'product_title': 'AGPtek® 10 Isolated Output 9V 12V 18V Guitar Pedal Board Power Supply Effect Pedals with Isolated Short Cricuit / Overcurrent Protection',
 'product_category': 'Musical Instruments',
 'star_rating': 3,
 'helpful_votes': 0,
 'total_votes': 1,
 'vine': 'N',
 'verified_purchase': 'N',
 'review_headline': 'Three Stars',
 'review_body': 'Works very good, but induces ALOT of noise.',
 'review_date': '2015-08-31'}

## To perform set intersections/unions efficiently :

## we first build data structures representing the set of items for each user and users for each item.

In [16]:
# Useful data structures

In [17]:
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)

In [18]:
itemNames = {}

In [19]:
for d in dataset: 
    user, item = d['customer_id'], d['product_id']
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    itemNames[item] = d['product_title']

## Jaccard similarity implementation 

### Jaccard(A, B) = |A intersaction B| / |A union B|

In [20]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer/denom

In [21]:
def mostSimilar(i):
    similarities = []
    users = usersPerItem[i]
    for i2 in usersPerItem:
        if i2 == i: continue
        sim = Jaccard(users, usersPerItem[i2])
        similarities.append((sim, i2))
    similarities.sort(reverse = True)
    return similarities[:10]

In [22]:
dataset[2]

{'marketplace': 'US',
 'customer_id': '6111003',
 'review_id': 'RIZR67JKUDBI0',
 'product_id': 'B0006VMBHI',
 'product_parent': '603261968',
 'product_title': 'AudioQuest LP record clean brush',
 'product_category': 'Musical Instruments',
 'star_rating': 3,
 'helpful_votes': 0,
 'total_votes': 1,
 'vine': 'N',
 'verified_purchase': 'Y',
 'review_headline': 'Three Stars',
 'review_body': 'removes dust. does not clean',
 'review_date': '2015-08-31'}

In [23]:
query = dataset[2]['product_id']

In [24]:
mostSimilar(query)

[(0.028446389496717725, 'B00006I5SD'),
 (0.01694915254237288, 'B00006I5SB'),
 (0.015065913370998116, 'B000AJR482'),
 (0.014204545454545454, 'B00E7MVP3S'),
 (0.008955223880597015, 'B001255YL2'),
 (0.008849557522123894, 'B003EIRVO8'),
 (0.008333333333333333, 'B0015VEZ22'),
 (0.00821917808219178, 'B00006I5UH'),
 (0.008021390374331552, 'B00008BWM7'),
 (0.007656967840735069, 'B000H2BC4E')]

In [25]:
itemNames[query]

'AudioQuest LP record clean brush'

In [26]:
[itemNames[x[1]] for x in mostSimilar(query)]

['Shure SFG-2 Stylus Tracking Force Gauge',
 'Shure M97xE High-Performance Magnetic Phono Cartridge',
 'ART Pro Audio DJPRE II Phono Turntable Preamplifier',
 'Signstek Blue LCD Backlight Digital Long-Playing LP Turntable Stylus Force Scale Gauge Tester',
 'Audio Technica AT120E/T Standard Mount Phono Cartridge',
 'Technics: 45 Adaptor for Technics 1200 (SFWE010)',
 'GruvGlide GRUVGLIDE DJ Package',
 'STANTON MAGNETICS Record Cleaner Kit',
 'Shure M97xE High-Performance Magnetic Phono Cartridge',
 'Behringer PP400 Ultra Compact Phono Preamplifier']

In [27]:
def mostSimilarFast(i):
    similarities = []
    users = usersPerItem[i]
    candidateItems = set()
    for u in users:
        candidateItems = candidateItems.union(itemsPerUser[u])
    for i2 in candidateItems:
        if i2 == i: continue
        sim = Jaccard(users, usersPerItem[i2])
        similarities.append((sim,i2))
    similarities.sort(reverse=True)
    return similarities[:10]

In [28]:
mostSimilarFast(query)

[(0.028446389496717725, 'B00006I5SD'),
 (0.01694915254237288, 'B00006I5SB'),
 (0.015065913370998116, 'B000AJR482'),
 (0.014204545454545454, 'B00E7MVP3S'),
 (0.008955223880597015, 'B001255YL2'),
 (0.008849557522123894, 'B003EIRVO8'),
 (0.008333333333333333, 'B0015VEZ22'),
 (0.00821917808219178, 'B00006I5UH'),
 (0.008021390374331552, 'B00008BWM7'),
 (0.007656967840735069, 'B000H2BC4E')]

In [29]:
#List of reviews per user and per item
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)

In [30]:
for d in dataset:
    user, item = d['customer_id'], d['product_id']
    reviewsPerUser[user].append(d) #Sorting records based on users for reviewsPerUser
    reviewsPerItem[item].append(d) #Sorting records based on items for reviewsPerItem

In [31]:
ratingMean = sum([d['star_rating'] for d in dataset]) / len(dataset)

In [32]:
ratingMean

4.251102772543146

In [33]:
def predictRating(user, item):
    ratings = []
    similarities = []
    
    for d in reviewsPerUser[user]:
        i2 = d['product_id']
        if i2 == item: continue
        ratings.append(d['star_rating'])
        similarities.append(Jaccard(usersPerItem[item], usersPerItem[i2]))
    
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings, similarities)]
        return sum(weightedRatings)/sum(similarities)
    
    else:
        # User hasn't rated any similar items
        return ratingMean    

In [34]:
dataset[1]

{'marketplace': 'US',
 'customer_id': '14640079',
 'review_id': 'RZSL0BALIYUNU',
 'product_id': 'B003LRN53I',
 'product_parent': '986692292',
 'product_title': 'Sennheiser HD203 Closed-Back DJ Headphones',
 'product_category': 'Musical Instruments',
 'star_rating': 5,
 'helpful_votes': 0,
 'total_votes': 0,
 'vine': 'N',
 'verified_purchase': 'Y',
 'review_headline': 'Five Stars',
 'review_body': 'Nice headphones at a reasonable price.',
 'review_date': '2015-08-31'}

In [35]:
u, i = dataset[1]['customer_id'], dataset[1]['product_id']

In [36]:
predictRating(u, i)

5.0

### Evaluate accuracy across the entire corpus

In [37]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences)/len(differences)

In [38]:
alwaysPredictMean = [ratingMean for d in dataset]

In [39]:
labels = [d['star_rating'] for d in dataset]

In [40]:
cfPredictions = [predictRating(d['customer_id'], d['product_id']) for d in dataset]

In [41]:
MSE(cfPredictions, labels)

1.6146130004291603

# Latent Factor Models - Bias only

In [42]:
N = len(dataset)
nUsers = len(reviewsPerUser)
nItems = len(reviewsPerItem)
users = list(reviewsPerUser.keys())
items = list(reviewsPerItem.keys())

### Alpha and Beta (userbiases)

In [45]:
#alpha and beta (userbiases) are parameters we'll fit. 
#This code sets their initial values (alpha to the mean rating, and beta_u/beta_i to zero)

alpha = ratingMean     #initial value to help gradient decent algorithm converge little bit quickly

userBiases = defaultdict(float)
itemBiases = defaultdict(float)

In [44]:
#Our prediction function in this case just implements the bias only model:

f(u,i) = alpha + beta_u + beta_i     #beta_u = user_bias, beta_i = item_bias

In [46]:
def prediction(user, item):
    return alpha + userBiases[user] + itemBiases[item]

In [47]:
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1 : nUsers+1]))
    itemBiases = dict(zip(items, theta[nUsers+1 : ]))

In [48]:
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(d['customer_id'], d['product_id']) for d in dataset]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    return cost

In [49]:
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(dataset)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    
    for d in dataset:
        u, i = d['customer_id'], d['product_id']
        pred = prediction(u, i)
        diff = pred - d['star_rating']
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff